# Saving to a SQL data base

This is an example of using a SQL database to store results from webscraping or API queries. The primary advantage of setting things up this way is that SQL databases are good for handling large structured databases and you don't need to hold much data in memory at any one given time.

In [1]:
from requests import get
import pandas as pd
import time
from bs4 import BeautifulSoup
import re
import sqlite3


In [4]:
def ap_parser(url):
    site = get(url)
    content = BeautifulSoup(site.content, "html.parser")
    timestamp = int(content.select_one('.Page-content bsp-timestamp').get('data-timestamp'))
    output = {
        'url' : site.url,
        'tags' : ', '.join([i.get('content') for i in content.select('meta[property="article:tag"]')]),
        'section' : ' '.join([i.get('content') for i in content.select('meta[property="article:section"]')]),
        'authors': ', '.join([i.get_text() for i in content.select('.Page-authors .Link')]),
        'article_text' : ' '.join([i.get_text() for i in content.select('.Page-content .RichTextBody p')]),
        'pubdate' : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp/1000)), # the timestamp on AP articles is a unix epoch
        'headline' : ' '.join([i.get_text() for i in content.select('.Page-headline')])
    }
    return output

# check if a table already exists, if it doesn't then create one
conn = sqlite3.connect('./ap_articles.db')

cur = conn.cursor()
print("checking for table")
try: 
    cur.execute('SELECT * from articles limit 1')
    print("table exists")
except sqlite3.OperationalError:
    print("No such table: creating")
    cur.execute('''
                CREATE TABLE articles(
                url text,
                tags text,
                section text,
                authors text,
                article_text text,
                pubdate text,
                headline text       
                )''')



From here, I'm just doing something similar to what we did in classwork 9-1: grabbing a sitemap, getting a list of articles and scraping each one. The key difference is that my results go into a SQL database called `ap_articles.db` and I only bother visiting a URL if there's not already a row for it in the data base. This means that I can come back over and over again to add new articles as they come in. 

In [57]:
sitemap = get('https://apnews.com/ap-sitemap.xml')

overall_sitemap = BeautifulSoup(sitemap.content, features="xml") 
# select all <loc> nodes
overall_nodes = overall_sitemap.select('loc')
# loop through the entire list and just get the link
sitemap_urls = [i.get_text() for i in overall_nodes]
sitemaps = [i for i in sitemap_urls if bool(re.search('2024|2025', i)) ]

article_urls = []

for i in sitemaps:
# get the sitemap
    response = get(i)
    # parse the content as an XML document
    sitemap= BeautifulSoup(response.content, features="xml") # note the features = 'xml' option!
    # select all <loc> nodes
    url_nodes = sitemap.select('loc')
    urls = [i.get_text() for i in url_nodes]
    article_urls.extend([i for i in urls if bool(re.search("/article/", i)) ])
    time.sleep(.1)
len(article_urls)

Searching for inflation related articles:

In [70]:
articles_list = [i for i in article_urls if bool(re.search(r"inflation", i)) ]
len(articles_list)

534

If new ones exist, then add them, otherwise, skip: 

In [ ]:
for url in articles_list:
    article_count = pd.read_sql(f"SELECT COUNT(*) as article_count from articles  where url = '{url}'", con=conn)
    if article_count['article_count'][0]==0:
        print("not found, scraping article", end='\r')
        try:
            article = ap_parser(url)
        except: 
            print("error encountered")
        finally:
            pd.DataFrame(article, index=[0]).to_sql(con = conn, name='articles', if_exists='append', index=False)
            time.sleep(1)
    else:
        print('article found', end='\r')

How many articles do we have now?

In [36]:
pd.read_sql("SELECT COUNT(*) from articles", con=conn)

,COUNT(*)
0,210


When you're done, close the connection to the SQL database:

In [71]:
conn.close()